Chelsea Jaculina

DATA 255 Assignment #5

October 13, 2025

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Build Deep Neural Network

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch.optim as optim

class MNISTDataset(Dataset):
    def __init__(self, csv_file):
        data = pd.read_csv(csv_file).values
        self.labels = torch.tensor(data[:, 0], dtype=torch.long)
        images = data[:, 1:].reshape(-1, 28, 28) / 255.0
        images = np.pad(images, ((0, 0), (2, 2), (2, 2)))  # pad to 32x32
        self.images = torch.tensor(images[:, None, :, :], dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# convolution Layer
class MyConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # weights and biases
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size) * 0.1)
        self.bias = nn.Parameter(torch.zeros(out_channels))

    def forward(self, x):
        B, C, H, W = x.shape
        # extract patches using unfold
        x_unfold = F.unfold(x, kernel_size=self.kernel_size)  # [B, C*k*k, L]
        w_flat = self.weight.view(self.out_channels, -1)      # [out_channels, C*k*k]
        out = w_flat @ x_unfold + self.bias[:, None]          # [out_channels, L]
        out_H = H - self.kernel_size + 1
        out_W = W - self.kernel_size + 1
        out = out.view(B, self.out_channels, out_H, out_W)
        return out


# average pooling layer
class MyAvgPool2d(nn.Module):
    def __init__(self, kernel_size=2, stride=2):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, x):
        B, C, H, W = x.shape
        # extract sliding windows
        x_unfold = F.unfold(x, kernel_size=self.kernel_size, stride=self.stride)
        x_unfold = x_unfold.view(B, C, self.kernel_size * self.kernel_size, -1)
        out = x_unfold.mean(dim=2)  # average pooling
        out_H = (H - self.kernel_size) // self.stride + 1
        out_W = (W - self.kernel_size) // self.stride + 1
        return out.view(B, C, out_H, out_W)


# fully connected linear layer
class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features) * 0.1)
        self.bias = nn.Parameter(torch.zeros(out_features))

    def forward(self, x):
        return x @ self.weight.T + self.bias

# LeNet-5 Network
class LeNet5Custom(nn.Module):
    def __init__(self):
        super().__init__()
        # conv1: 32x32 -> 28x28x6
        self.conv1 = MyConv2d(1, 6, 5)
        self.pool1 = MyAvgPool2d(2, 2)

        # conv2: 14x14x6 -> 10x10x16
        self.conv2 = MyConv2d(6, 16, 5)
        self.pool2 = MyAvgPool2d(2, 2)

        # fully connected Layers
        self.fc1 = MyLinear(16 * 5 * 5, 120)
        self.fc2 = MyLinear(120, 84)
        self.fc3 = MyLinear(84, 10)

    def forward(self, x):
        # layer 1: Conv -> Tanh -> AvgPool
        x = torch.tanh(self.pool1(self.conv1(x)))

        # layer 2: Conv -> Tanh -> AvgPool
        x = torch.tanh(self.pool2(self.conv2(x)))

        # flatten
        x = x.view(x.size(0), -1)

        # fully connected layers
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        return self.fc3(x)  # logits

print("Convolution Layer defined successfully")
print("Average Pooling Layer defined successfully")
print("Fully Connected Layer defined successfully")
print("LeNet-5 Network defined successfully")

Convolution Layer defined successfully
Average Pooling Layer defined successfully
Fully Connected Layer defined successfully
LeNet-5 Network defined successfully


In [4]:
# accuracy helper function
def calculate_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

print("Accuracy function defined successfully")

# training the custom LeNet-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load data
train_dataset = MNISTDataset("/content/drive/MyDrive/MSDA 2024-2026/04 Fall 2025/DATA 255 - Deep Learning/mnist_train.csv")
test_dataset  = MNISTDataset("/content/drive/MyDrive/MSDA 2024-2026/04 Fall 2025/DATA 255 - Deep Learning/mnist_test.csv")

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Train data loaded successfully")
print("Test data loaded successfully")


Accuracy function defined successfully
Train data loaded successfully
Test data loaded successfully


## 2. Training

In [5]:
import time

# initialize model, loss, and optimizer
model = LeNet5Custom().to(device)
print("Initialize LeNet-5 model successfully")
criterion = nn.CrossEntropyLoss()
print("Initialize Loss Function successfully")
optimizer = optim.SGD(model.parameters(), lr=0.01)
print("Initialize Optimizer successfully\n")

# training loop
num_epochs = 10
total_start = time.time()

for epoch in range(num_epochs):
    epoch_start = time.time()

    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad() # clear previous gradients
        outputs = model(imgs) # forward pass
        loss = criterion(outputs, labels)
        loss.backward() # backward pass (autograd)
        optimizer.step() # update weights

    # measure accuracy
    train_acc = calculate_accuracy(model, train_loader)
    test_acc = calculate_accuracy(model, test_loader)

    epoch_end = time.time()
    epoch_time = epoch_end - epoch_start

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Loss: {loss.item():.4f} | "
          f"Train Acc: {train_acc:.4f} | "
          f"Test Acc: {test_acc:.4f} | "
          f"Time: {epoch_time:.2f} sec")

Initialize LeNet-5 model successfully
Initialize Loss Function successfully
Initialize Optimizer successfully

Epoch 1/10 | Loss: 0.2115 | Train Acc: 0.9524 | Test Acc: 0.9514 | Time: 9.17 sec
Epoch 2/10 | Loss: 0.0495 | Train Acc: 0.9674 | Test Acc: 0.9665 | Time: 10.30 sec
Epoch 3/10 | Loss: 0.0489 | Train Acc: 0.9720 | Test Acc: 0.9735 | Time: 10.21 sec
Epoch 4/10 | Loss: 0.0406 | Train Acc: 0.9754 | Test Acc: 0.9762 | Time: 10.80 sec
Epoch 5/10 | Loss: 0.1405 | Train Acc: 0.9754 | Test Acc: 0.9768 | Time: 8.96 sec
Epoch 6/10 | Loss: 0.0365 | Train Acc: 0.9787 | Test Acc: 0.9795 | Time: 10.22 sec
Epoch 7/10 | Loss: 0.1315 | Train Acc: 0.9785 | Test Acc: 0.9795 | Time: 10.58 sec
Epoch 8/10 | Loss: 0.0129 | Train Acc: 0.9805 | Test Acc: 0.9827 | Time: 9.38 sec
Epoch 9/10 | Loss: 0.0075 | Train Acc: 0.9812 | Test Acc: 0.9822 | Time: 10.08 sec
Epoch 10/10 | Loss: 0.0111 | Train Acc: 0.9825 | Test Acc: 0.9816 | Time: 10.11 sec
